# Алгоритмы планирования: MCTS

<img src="mcts.png">

Импортируем необходимые библиотеки:

In [2]:
import random

import numpy as np

from draw_graph import draw_graph
from go_env import GoEnv

Посмотри на наше окружение -- игру Го.

In [3]:
g = GoEnv(5)
g.reset()

for _ in range(5):
    g.step(np.random.choice(g.get_possible_actions()))

g.render()

Move:   5  Komi: 0.0  Handicap: 0  Captures B: 0 W: 0
      A B C D E  
    +-----------+
  5 | . . . B W |
  4 | . . . . . |
  3 | . . . . B |
  2 | . . . B). |
  1 | . W . . . |
    +-----------+


### Задание 1. Заполните пропуски в реализации алгоритма UCT
За основу мы возьмем реализацию из статьи: MCTS Survey, которую мы частично рассмотрели на прошлом семинаре. 
<img src="uct.png">

In [9]:
class UCT:
    class Node:
        def __init__(self, board, state, parent):
            self.board = board
            self.wins = 0
            self.games = 0
            self.state = state
            self.children = {}
            self.parent = parent
            self.view = "\n".join(self.board.__repr__().decode().replace("O", "W").replace("X", "B").split("\n")[1:])
            self.view = str(self.board.__hash__()) + "\n" + self.view.replace("(", "").replace(")", "")

        def __hash__(self):
            return self.board.__hash__()

        def __str__(self):
            return self.view + "\n    " + str(self.wins) + "/" + str(self.games)

    def __init__(self):
        self.root = None
        self.all_nodes = {}

    def save_tree(self, filename="out"):
        draw_graph(file_name=filename, graph=self.get_graph(self.root))

    def get_graph(self, node, edges=None):
        if node is None:
            node = self.root
        if edges is None:
            edges = []
        for action in node.children:
            edges.append((node.__str__(), node.children[action].__str__()))
            self.get_graph(node=node.children[action], edges=edges)
        return edges

    def get_action(self, env: GoEnv):
        if env.get_state() in self.all_nodes and self.all_nodes[env.get_state()].children:
            node = self.all_nodes[env.get_state()]
            return self.best_child(node, cp=0)
        else:
            return np.random.choice(env.get_possible_actions())

    def search_uct(self, env, budget=200):

        # создаем v0
        state = env.reset()
        if self.root is None:
            self.root = self.Node(env.board, parent=None, state=env.get_state())
            self.all_nodes[state] = self.root
        v0 = self.root
        for _ in range(budget):
            env.reset()
            # вызываем tree_policy
            vl = self.tree_policy(v0, env)
            # вызываем default policy
            result = self.default_policy(env)
            # backup
            self.backup(vl, result)

    def tree_policy(self, vl, env: GoEnv):
        while not env.done:
            # реализуем then ветку алгоритма (not fully expanded)
            ###### Your code here ##########
            raise NotImplementedError
            ################################
            # else ...
            action = self.best_child(vl)
            env.step(action)
            vl = vl.children[action]
        return vl

    def expand(self, vl, env):
        action = None
        # выбираем подходящее действие
        ###### Your code here ##########
        raise NotImplementedError
        ################################
        # получаем следующее состояние и возвращаем вершину
        state, r, done, _ = env.step(action)
        if state not in self.all_nodes:
            vl.children[action] = self.Node(board=env.board, parent=vl, state=env.get_state())
            self.all_nodes[vl.children[action].state] = vl.children[action]
        else:
            vl.children[action] = self.all_nodes[state]
        return vl.children[action]

    @staticmethod
    def best_child(parent: Node, cp=1 / np.sqrt(2)):
        """выбираем лучшее действие, согласно алгоритму UCT"""
        ###### Your code here ##########
        raise NotImplementedError
        ################################
        return best

    @staticmethod
    def default_policy(env):
        if env.done:
            # если игра завершилась возвращем вознаграждение
            return env.get_reward()
        else:
            # реазилуем случайную стратегию, возвращаем вознаграждение (не забывайте про то, чей сейчас ход!)
            ###### Your code here ##########
            raise NotImplementedError
            ################################
            return reward

    @staticmethod
    def backup(vl, result):
        # реализуем backpropagation для двух игроков!
        ###### Your code here ##########
        raise NotImplementedError
        ################################

In [12]:
uct10 = UCT()
uct10.search_uct(GoEnv(5), budget=10)

Мы можем нарисовать полученное дерево, с помощью функции save_tree и библиотеки pygraphviz

In [13]:
# !sudo apt-get install python-dev graphviz libgraphviz-dev pkg-config
# !pip3 install pygraphviz
uct10 = UCT()
uct10.search_uct(GoEnv(2), budget=10)
uct10.save_tree("uct10go2")

<img src="uct10go2.png">

### Задание 2. Сравните двух UCT агентов с разным временем, затраченным на обучение. Используйте мтод get_action()

### Задание 3. Добавьте агентам возможность проводить N шагов поиска во время игры и сравните агентов